In [7]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.3'
spark_version = 'spark-3.4.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Print the content of the Spark directory to verify installation
!ls /content/$SPARK_VERSION-bin-hadoop3

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
bin   data	j

In [8]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
home_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [11]:
# 2. Create a temporary view of the DataFrame.

home_df.createOrReplaceTempView('home_data')

In [12]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

avg_price_4_bedroom = spark.sql("SELECT ROUND(AVG(price),2), YEAR(date) from home_data where bedrooms == 4 group by YEAR(date) ORDER BY YEAR(date) desc")
avg_price_4_bedroom.show()

+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|           301819.44|      2021|
|           298353.78|      2020|
|            300263.7|      2019|
+--------------------+----------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_price_3_bed_3_bath = spark.sql("SELECT ROUND(AVG(price),2), date_built from home_data where bedrooms == 3 AND bathrooms == 3 group by date_built ORDER BY date_built desc")
avg_price_3_bed_3_bath.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           292676.79|      2017|
|           290555.07|      2016|
|            288770.3|      2015|
|           290852.27|      2014|
|           295962.27|      2013|
|           293683.19|      2012|
|           291117.47|      2011|
|           292859.62|      2010|
+--------------------+----------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_3_bed_3_bath_2_floors = spark.sql("SELECT ROUND(AVG(price),2), date_built from home_data where bedrooms == 3 AND bathrooms == 3 AND floors == 2 AND sqft_living >= 2000 group by date_built ORDER BY date_built desc")
avg_price_3_bed_3_bath_2_floors.show()


+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           280317.58|      2017|
|            293965.1|      2016|
|           297609.97|      2015|
|           298264.72|      2014|
|           303676.79|      2013|
|           307539.97|      2012|
|           276553.81|      2011|
|           285010.22|      2010|
+--------------------+----------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, ROUND(AVG(price),2) from home_data group by view having ROUND(AVG(price),2) >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))




+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.9816536903381348 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `home_sales` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 12;
'CacheTable [home_sales], false, false
+- 'UnresolvedRelation [home_sales], [], false


In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_data')

True

In [20]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql("SELECT view, ROUND(AVG(price),2) from home_data group by view having ROUND(AVG(price),2) < 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))




+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  15|           284907.04|
|  11|           280356.07|
|  29|           283881.72|
|  42|           289225.45|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  16|           291990.83|
|  35|           281767.41|
|   0|           285069.21|
|  47|           292925.62|
|  43|           282606.92|
|   5|           278096.94|
|  31|           287988.84|
|  18|           287532.36|
|  27|            281702.6|
+----+--------------------+
only showing top 20 rows

--- 0.5871882438659668 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.parquet('home_parquet', mode='overwrite')

In [22]:
# 11. Read the parquet formatted data.
parquet_home_df = spark.read.parquet('home_parquet')

In [23]:
# 12. Create a temporary table for the parquet data.
parquet_home_df.createOrReplaceTempView('parquet_temp_home')

In [24]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
spark.sql("SELECT view, ROUND(AVG(price),2) from parquet_temp_home group by view having ROUND(AVG(price),2) < 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))




+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   7|           288929.09|
|  15|           284907.04|
|  11|           280356.07|
|  29|           283881.72|
|  42|           289225.45|
|   3|           284314.53|
|  30|           281085.62|
|  34|           286124.07|
|   8|           279099.78|
|  22|           284908.42|
|  28|           285474.25|
|  16|           291990.83|
|  35|           281767.41|
|   0|           285069.21|
|  47|           292925.62|
|  43|           282606.92|
|   5|           278096.94|
|  31|           287988.84|
|  18|           287532.36|
|  27|            281702.6|
+----+--------------------+
only showing top 20 rows

--- 0.7569475173950195 seconds ---


In [25]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_data")

DataFrame[]

In [26]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_data')


False